In [2]:
import pandas as pd

# 1. 检测缺失值
# 加载数据
data = pd.read_excel("数据.xlsx")

# 检测缺失值
missing_values = data.isnull().sum()  # 统计每列的缺失值数量
print("缺失值统计：\n", missing_values)

# 2. 检测异常值
# 以数值列为例
numerical_columns = data.select_dtypes(include=['int', 'float']).columns

# 使用箱线图检测异常值
def detect_outliers(data):
    outliers = pd.DataFrame(columns=['feature', 'number_of_outliers'])
    for column in numerical_columns:
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = outliers.append({'feature': column, 'number_of_outliers': data[(data[column] < lower_bound) | (data[column] > upper_bound)].shape[0]}, ignore_index=True)
    return outliers

outliers = detect_outliers(data)
print("异常值统计：\n", outliers)


缺失值统计：
 指标          0
粮食产量        0
农作物受灾面积     0
平均温度        0
农业总产值       0
平均降雨量       0
农业机械总动力     0
农药使用量       0
第一产业就业人员    0
dtype: int64
异常值统计：
     feature number_of_outliers
0        指标                  0
1      粮食产量                  1
2   农作物受灾面积                  1
3      平均温度                  0
4     农业总产值                  0
5     平均降雨量                  0
6   农业机械总动力                  0
7     农药使用量                  0
8  第一产业就业人员                  0


In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler

# 1. 加载数据
data = pd.read_excel("数据.xlsx")

# 2. 检测并替换异常值
def replace_outliers(data):
    numerical_columns = data.select_dtypes(include=['int', 'float']).columns
    for column in numerical_columns:
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        data[column] = data[column].mask((data[column] < lower_bound) | (data[column] > upper_bound), data[column].median())
    return data

data = replace_outliers(data)

# 3. 使用插值法处理缺失值
imputer = SimpleImputer(strategy="linear")  # 这里使用线性插值，也可以选择其他插值方法
data_filled = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# 4. 特征缩放
scaler = RobustScaler()  # 使用RobustScaler来缩放数据，对异常值更加鲁棒
data_scaled = scaler.fit_transform(data_filled)

# 将缩放后的数据放回DataFrame
data_filled = pd.DataFrame(data_scaled, columns=data_filled.columns)

# 输出处理后的数据
print(data_filled.head())
